In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer
from sklearn.decomposition import PCA

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
datos_combi  = pd.read_csv(path_archivo)

In [3]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)
datos_combi

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.196350,1.047198,0.999998,0.002000,3.910766e-07,8.072690,13.084104,27.374015,33.406144,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.196350,1.047198,0.999987,0.005138,1.003534e-06,8.072280,13.084550,27.402665,33.401889,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.196350,1.047198,0.999966,0.008275,1.614694e-06,8.071871,13.084995,27.431316,33.397652,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.196350,1.047198,0.999935,0.011413,2.224556e-06,8.071464,13.085440,27.459968,33.393435,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.196350,1.047198,0.999894,0.014550,2.833116e-06,8.071059,13.085883,27.488622,33.389236,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52495,1.555104,1.570796,3.141593,0.015691,0.999877,2.657355e-01,1.000000,1.095280,1.689990,3.591959,...,253.186306,256.177121,260.447769,266.168628,266.903370,266.903370,267.005409,275.877985,275.877985,276.863432
52496,1.558242,1.570796,3.141593,0.012554,0.999921,2.152209e-01,1.000000,1.087282,1.677534,4.013426,...,312.142046,316.286467,321.456500,328.422287,329.221352,329.295933,329.295933,340.346872,340.346872,341.514727
52497,1.561380,1.570796,3.141593,0.009417,0.999956,1.635543e-01,1.000000,1.078852,1.664080,4.723116,...,410.150304,416.173150,422.841844,431.875484,432.621470,432.981372,432.981372,447.483081,447.483081,448.953224
52498,1.564517,1.570796,3.141593,0.006279,0.999980,1.105994e-01,1.000000,1.069906,1.649423,6.154539,...,605.765944,615.389302,625.053291,638.210281,638.865702,639.781796,639.781796,661.166268,661.166268,663.238317


In [4]:
dat_copy = datos_combi.copy()
operacion = lambda x: np.log(x)
for i in range(10):
    key_mod = "eig_" + str(i+1)
    prev_key = "eig_" + str(i)
    if i == 0:  
        dat_copy[key_mod] = 1/datos_combi[key_mod]
    else:
        dat_copy[key_mod] = datos_combi[prev_key]/datos_combi[key_mod]
    #fin if 
    #dat_copy[key_mod] = np.array(list(map(operacion, dat_copy[key_mod])))
#fin for
#dat_copy["mu_sobre_K"] = np.tan(dat_copy["phi_K"])
datos_combi = dat_copy.copy()


In [5]:
datos_combi.head()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.19635,1.047198,0.999998,0.002000,3.910766e-07,0.123874,0.616985,0.477975,0.819431,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.19635,1.047198,0.999987,0.005138,1.003534e-06,0.123881,0.616932,0.477492,0.820393,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.19635,1.047198,0.999966,0.008275,1.614694e-06,0.123887,0.616880,0.477009,0.821355,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.19635,1.047198,0.999935,0.011413,2.224556e-06,0.123893,0.616828,0.476528,0.822316,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.19635,1.047198,0.999894,0.014550,2.833116e-06,0.123899,0.616776,0.476047,0.823278,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198


In [6]:
N_eig = 5
orden_pol = 6
for i in range(N_eig):
    col = "eig_" + str(i+1)
    #datos_combi[col] = np.log(datos_combi[col])
#fin for 
datos_combi = datos_combi.copy()
datos_combi.head()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.19635,1.047198,0.999998,0.002000,3.910766e-07,0.123874,0.616985,0.477975,0.819431,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.19635,1.047198,0.999987,0.005138,1.003534e-06,0.123881,0.616932,0.477492,0.820393,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.19635,1.047198,0.999966,0.008275,1.614694e-06,0.123887,0.616880,0.477009,0.821355,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.19635,1.047198,0.999935,0.011413,2.224556e-06,0.123893,0.616828,0.476528,0.822316,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.19635,1.047198,0.999894,0.014550,2.833116e-06,0.123899,0.616776,0.476047,0.823278,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198


In [7]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [8]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/"):
    cols_eig = list(map(lambda x: "eig_" + str(x+1), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred_" + nombre
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel("lambda moño")
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [9]:
def graficar_todas(n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(datos_combi.iloc[start:end], target, n_eig, preds_fin, op)
    #fin for 
#fin función

In [10]:
target = "phi_K"
graficar_todas(N_eig, target)

In [11]:
#Vamos a entrenar el modelo con TODOS los datos combinatoriales. Nuestros datos de test serán aleatorios. 
features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
features_dim = ["eta", "beta"]
features = features_dim + features_eig
X_train = datos_combi[features]
y_train = datos_combi[target]
print(features)

['eta', 'beta', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5']


In [12]:
X_train.head()

,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5
0,0.19635,1.047198,0.123874,0.616985,0.477975,0.819431,0.354728
1,0.19635,1.047198,0.123881,0.616932,0.477492,0.820393,0.354651
2,0.19635,1.047198,0.123887,0.616880,0.477009,0.821355,0.354574
3,0.19635,1.047198,0.123893,0.616828,0.476528,0.822316,0.354497
4,0.19635,1.047198,0.123899,0.616776,0.476047,0.823278,0.354420


In [13]:
def get_metrics(X, y, model):
    y_gorro = model.predict(X)
    R2 = r2_score(y, y_gorro)
    RMSE = root_mean_squared_error(y, y_gorro)
    MAE = mean_absolute_error(y, y_gorro)
    return {"R2": R2, "RMSE": RMSE, "MAE": MAE}
#fin función

class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, cols_tr, deg = 3):
        self.deg = deg
        self.cols_tr = cols_tr
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X):
        datos = X.copy()
        cols_transformar = self.cols_tr
        poly = PolynomialFeatures(degree=self.deg, include_bias=False)
        poly_data = poly.fit_transform(datos[cols_transformar])
        poly_feature_names = poly.get_feature_names_out(input_features=cols_transformar)
        datos_finales = pd.DataFrame(poly_data, columns=poly_feature_names)
        return datos_finales
    #fin transform
#fin class

In [14]:
def crear_pipeline(modelo, grado, nombre_modelo, cols_tr):
    pipeline = Pipeline([('custom-transformer', TransformadorDeDatos(cols_tr, deg = grado)),
            #('power-transformer', PowerTransformer(method = "box-cox", standardize = True)), 
            (nombre_modelo, modelo)])
    return pipeline
#fin función

In [15]:
pipeline = crear_pipeline(LinearRegression(), orden_pol, "re_lineal_simple", features)

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('custom-transformer',
                 TransformadorDeDatos(cols_tr=['eta', 'beta', 'eig_1', 'eig_2',
                                               'eig_3', 'eig_4', 'eig_5'],
                                      deg=6)),
                ('re_lineal_simple', LinearRegression())])

In [17]:
metrs = get_metrics(X_train, y_train, pipeline)

In [18]:
print(metrs)

{'R2': 0.999015070523925, 'RMSE': 0.014212698067223518, 'MAE': 0.0064900267434495566}


In [19]:
y_gorro_train = pipeline.predict(X_train)

In [20]:
graficar_todas(N_eig, target, preds={"y": y_gorro_train, "X": X_train})

In [21]:
path_test = "../input_data/uni_52500_1.csv"
datos_test_full = pd.read_csv(path_test)

In [22]:
datos_test_full.head()

,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.513461,0.520834,2.462206,0.034106,1.764647,3.465653,6.446372,9.886191,11.562449,14.789919,...,1073.047116,1082.426697,1102.042351,1151.160382,1151.722711,1193.775156,1208.780911,1219.344722,1224.141007,1225.155069
1,1.453175,0.904978,0.678690,0.008021,4.361191,8.879599,15.346436,18.378833,21.309113,31.472544,...,8589.366731,8740.973305,8745.939505,8813.854849,8957.763890,9360.001209,9441.588347,9498.228970,9600.635186,9771.157762
2,0.305289,0.786018,1.750694,0.052944,1.738172,2.914895,6.154208,7.451840,9.088756,11.498754,...,476.043469,481.645629,493.045906,494.290833,498.284527,502.342678,527.075912,542.669933,547.161699,552.743847
3,0.116401,0.508804,1.561187,0.002957,3.871294,4.498427,7.313661,16.944009,23.539851,25.482868,...,3110.277601,3245.658599,3246.520316,3306.172525,3374.102154,3633.979333,3663.903530,3699.166178,3719.122191,3730.560386
4,0.235985,1.262090,2.113612,0.269728,1.356214,2.259893,4.000866,4.099376,4.148445,5.161244,...,85.275208,85.336410,86.472296,87.721344,88.834995,92.506316,94.098947,94.165783,95.719851,96.214931


In [23]:
test_copy = datos_test_full.copy()
operacion = lambda x: np.log(x)
for i in range(10):
    key_mod = "eig_" + str(i+1)
    prev_key = "eig_" + str(i)
    if i == 0:  
        test_copy[key_mod] = 1/datos_test_full[key_mod]
    else:
        test_copy[key_mod] = datos_test_full[prev_key]/datos_test_full[key_mod]
    #fin if 
#fin for
datos_test = test_copy.copy()
datos_test.head()

,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.513461,0.520834,2.462206,0.034106,0.566686,0.509182,0.537613,0.652058,0.855026,0.781779,...,1073.047116,1082.426697,1102.042351,1151.160382,1151.722711,1193.775156,1208.780911,1219.344722,1224.141007,1225.155069
1,1.453175,0.904978,0.678690,0.008021,0.229295,0.491147,0.578610,0.835006,0.862487,0.677070,...,8589.366731,8740.973305,8745.939505,8813.854849,8957.763890,9360.001209,9441.588347,9498.228970,9600.635186,9771.157762
2,0.305289,0.786018,1.750694,0.052944,0.575317,0.596307,0.473643,0.825864,0.819897,0.790412,...,476.043469,481.645629,493.045906,494.290833,498.284527,502.342678,527.075912,542.669933,547.161699,552.743847
3,0.116401,0.508804,1.561187,0.002957,0.258312,0.860588,0.615072,0.431637,0.719801,0.923752,...,3110.277601,3245.658599,3246.520316,3306.172525,3374.102154,3633.979333,3663.903530,3699.166178,3719.122191,3730.560386
4,0.235985,1.262090,2.113612,0.269728,0.737347,0.600123,0.564851,0.975970,0.988172,0.803769,...,85.275208,85.336410,86.472296,87.721344,88.834995,92.506316,94.098947,94.165783,95.719851,96.214931


In [24]:
for i in range(N_eig):
    col = "eig_" + str(i+1)
    #datos_test[col] = np.log(datos_test[col])
#fin for 
datos_test = datos_test.copy()
datos_test.head()

,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.513461,0.520834,2.462206,0.034106,0.566686,0.509182,0.537613,0.652058,0.855026,0.781779,...,1073.047116,1082.426697,1102.042351,1151.160382,1151.722711,1193.775156,1208.780911,1219.344722,1224.141007,1225.155069
1,1.453175,0.904978,0.678690,0.008021,0.229295,0.491147,0.578610,0.835006,0.862487,0.677070,...,8589.366731,8740.973305,8745.939505,8813.854849,8957.763890,9360.001209,9441.588347,9498.228970,9600.635186,9771.157762
2,0.305289,0.786018,1.750694,0.052944,0.575317,0.596307,0.473643,0.825864,0.819897,0.790412,...,476.043469,481.645629,493.045906,494.290833,498.284527,502.342678,527.075912,542.669933,547.161699,552.743847
3,0.116401,0.508804,1.561187,0.002957,0.258312,0.860588,0.615072,0.431637,0.719801,0.923752,...,3110.277601,3245.658599,3246.520316,3306.172525,3374.102154,3633.979333,3663.903530,3699.166178,3719.122191,3730.560386
4,0.235985,1.262090,2.113612,0.269728,0.737347,0.600123,0.564851,0.975970,0.988172,0.803769,...,85.275208,85.336410,86.472296,87.721344,88.834995,92.506316,94.098947,94.165783,95.719851,96.214931


In [25]:
datos_test = datos_test.dropna()

In [26]:
problematic_rows = datos_test[(~np.isfinite(datos_test)).any(axis=1) | (datos_test.abs() > np.finfo(np.float64).max).any(axis=1)]
problematic_indices = problematic_rows.index
datos_test = datos_test.drop(index=problematic_indices)
print(len(datos_test))

52492


In [27]:
X_test = datos_test[features]
y_test = datos_test[target]

In [28]:
X_test.head()

,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5
0,0.520834,2.462206,0.566686,0.509182,0.537613,0.652058,0.855026
1,0.904978,0.678690,0.229295,0.491147,0.578610,0.835006,0.862487
2,0.786018,1.750694,0.575317,0.596307,0.473643,0.825864,0.819897
3,0.508804,1.561187,0.258312,0.860588,0.615072,0.431637,0.719801
4,1.262090,2.113612,0.737347,0.600123,0.564851,0.975970,0.988172


In [29]:
print(len(X_test), len(y_test))

52492 52492


In [30]:
y_gorro_test = pipeline.predict(X_test)

In [31]:
metrics_test = get_metrics(X_test, y_test, pipeline)

In [32]:
print(metrics_test)

{'R2': -145994.35537688344, 'RMSE': 173.3189278591441, 'MAE': 3.057578231488912}


In [33]:
print(np.c_[y_test, y_gorro_test])

[[ 0.51346063  0.36792991]
 [ 1.45317461  1.70069934]
 [ 0.30528948  0.31160237]
 ...
 [ 0.16049669  0.1272777 ]
 [ 1.18994595 -5.08872403]
 [ 0.89186676 -6.22767863]]
